# **Notebook Setup: Import Libraries**

In [ ]:
# Importing the required libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Fallback api to fetch stock data
import yfinance

# **Setup: Parameter Adjustment**

In [ ]:
#Variables & Parameters to adjust below

#Data Loading
stock = "MEBL.KA"
split = 0.8 #Test data is automatically calculated based on this, 0.8 here means 80% Train & 20% test.

#Holidays Setup
country = "PAK" #Uses the built-in holidays list by country code, good enough for baseline.
use_custom_holidays = True #Sets to use the custom defined holidays, make sure to change them according to your needs.

# **Step 1: Data Loading**

In [ ]:
try:
  # Attempt to load data from local directory first
  # Note: Prophet expects 2 columns: 'ds' (date) and 'y' (value to forecast)
  # If the data is not in this format, it will need to be transformed. 
  df = pd.read_csv(f'../data/{stock}.csv')
  print(f"Successfully loaded data for {stock} from local directory '../data/'.")

except Exception as e:
  print(f"Failed to load data from local directory: {e}")
  print(f"Attempting to fetch data for {stock} using yfinance...")

  try:
    # Tries to load data from yfinance api since Local methood failed.
    stock_data = yfinance.Ticker(stock)
    df = stock_data.history(period="max")

    # yfinance uses 'Date' as index, reset to a column for consistency
    df = df.reset_index()
    df.rename(columns={'Date': 'date', 'Close': 'close', 'Volume': 'volume', 'Open': 'open', 'High': 'high', 'Low': 'low'}, inplace=True)

    # Prophet requires 'ds' and 'y' columns
    df2 = pd.DataFrame()
    df2['ds'] = df['date'].dt.date
    df2['y'] = df['close']
    df = df2.copy()

    if not df.empty:
      print(f"Successfully fetched data for {stock} using yfinance.")
    else:
      print("ERROR: No data could be loaded for analysis. Please check the ticker symbol matches yfinance")

  except Exception as e:
    print(f"Failed to fetch data for {stock} using yfinance: {e}")
    print("Both methods failed. Please ensure the data file is in '../data/'")
    print(f"or that '{stock}' is a valid ticker symbol on yfinance.")
    df = None # Ensure df is None if both methods fail

In [ ]:
# Displays the first 5 rows of the loaded dataset, check to make sure it's correct
if df is not None:
  print(df.tail())
else:
  print("Data could not be loaded using either method.")

# **Step 2: Data Pre-Processing**

In [ ]:
# Extract the date and closing price columns
ds = df['ds']
y = df['y']

# Calculate the number of data points for training and testing
number_of_data_points = len(df)
train_data_size = math.ceil(number_of_data_points * split)

# Split the data into training and testing sets
df_train = pd.DataFrame(data={'ds': ds[:train_data_size], 'y': y[:train_data_size]})
df_test = pd.DataFrame(data={'ds': ds[train_data_size:], 'y': y[train_data_size:]})

# **Step 3: Adding holidays to feed to the model, gives better predictions**

In [ ]:
#Custom Holidays
New_Years_Eve = pd.DataFrame({
    'holiday': 'New Years Eve',
    'ds': pd.to_datetime(['2013-12-31', '2014-12-31','2015-12-31','2016-12-31','2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-12-31', '2022-12-31',]),
    'lower_window': 0,
    'upper_window': 1,
})

Bank_Holiday = pd.DataFrame({
    'holiday': 'Bank Holiday',
    'ds': pd.to_datetime(['2013-07-01', '2014-07-01','2015-07-01','2016-07-01','2017-07-01','2018-07-01','2019-07-01','2020-07-01','2021-07-01', '2022-07-01',]),
    'lower_window': 0,
    'upper_window': 0,
})

Defence_Day = pd.DataFrame({
    'holiday': 'Defence Day',
    'ds': pd.to_datetime(['2013-09-01', '2014-09-01','2015-09-01','2016-09-01','2017-09-01','2018-09-01','2019-09-01','2020-09-01','2021-09-01', '2022-09-01',]),
    'lower_window': 0,
    'upper_window': 0,
})

# Combines all the holiday DataFrames into a single DataFrame if
# user_custom_holidays is True
if use_custom_holidays == True:
  holidays = pd.concat((New_Years_Eve, Bank_Holiday, Defence_Day))
else:
  holidays = None

# **Step 4: Initialize model and fit training data**

In [ ]:
# Instantiate prophet
m = Prophet(holidays=holidays)
m.add_country_holidays(country_name=country)

# fit the training data
m.fit(df_train)

In [ ]:
#Use this to check which holidays were included in the forecast
m.train_holiday_names

# **Step 5: Make forecast**

In [ ]:
#Makes the forecast on the test data
forecast = m.predict(df_test)

# Outputs the last 5 values of the forecasted data, filtered for the specific columns.
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# **Step 6: Visualizing Graphs**

In [ ]:
#fig1 gives the standard plot chart, black like is the test data, the blue line with the spread is the forecasted data.
fig1 = m.plot(forecast)

#fig2 gives additional visual representations of things like trend or when holidays appeared.
fig2 = m.plot_components(forecast)

# **Step 7: Evaluate model accuray**

In [ ]:
# Evaluation metrics to measure model accuracy, uses MSE, MAE, and RMSE.
print("Mean Squared Error (MSE):", mean_squared_error(y_true = df_test["y"], y_pred = forecast['yhat']))
print("Mean Absolute Error (MAE):", mean_absolute_error(y_true = df_test["y"], y_pred = forecast['yhat']))
print("Root Mean Squared Error (RMSE):", np.sqrt(mean_squared_error(y_true = df_test["y"], y_pred = forecast['yhat'])))